In [ ]:
import apache_beam as beam
import json
import logging
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions


In [ ]:
project_id = "serverless-edem"
dataset_id = "edem_dataset"
table_id = "edem_table_v4"
subscription_id ="test_beam-sub"

In [ ]:
"""
Clean cache
rm -rf ~/.apache_beam
"""

In [10]:
import apache_beam as beam
import json
import logging
import sys
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions


logging.getLogger().setLevel(logging.INFO)


sys.argv = sys.argv[:1]

def decode_message(msg):
    
    output = msg.decode('utf-8')
    logging.info("New PubSub Message: %s", output)
    
    return json.loads(output)

pipeline_options = PipelineOptions()
pipeline_options.view_as(StandardOptions).streaming = True

with beam.Pipeline(options=pipeline_options) as p:
    (
        p 
            | "ReadFromPubSub" >> beam.io.ReadFromPubSub(subscription=f'projects/{project_id}/subscriptions/{subscription_id}')
            | "Decode msg" >> beam.Map(decode_message)
            | "Write to BigQuery" >> beam.io.WriteToBigQuery(
                table=f"{project_id}:{dataset_id}.{table_id}",
                schema='temp:INTEGER', 
                create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
            )
    )

INFO:root:Missing pipeline option (runner). Executing pipeline using the default runner: DirectRunner.
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with PrismRunner.
INFO:apache_beam.runners.worker.worker_pool_main:Listening for workers at localhost:56355
I0000 00:00:1738339945.272058  725254 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
INFO:apache_beam.runners.portability.prism_runner:Downloading prism binary from https://github.com/apache/beam/releases/download/v2.62.0/apache_beam-v2.62.0-prism-darwin-arm64.zip
INFO:apache_beam.utils.subprocess_server:Starting service with ('/Users/javierbrionesgomez/.apache_beam/cache/prism/bin/apache_beam-v2.62.0-prism-darwin-arm64' '--job_port' '56360' '--serve_http' 'False')
INFO:apache_beam.utils.subprocess_server:[2025-01-31T17:12:36.760677+01:00]  INFO  Serving JobManagement
INFO:apache_beam.utils.subprocess_server:* endpoint: localhost:56360
INFO:apache_beam.utils.subprocess_serve

In [ ]:
!pip show apache-beam